In [11]:
from tracemalloc import start
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, ConcatDataset
from datasets.dataset import *
# from models.Autoformer import *
from tqdm import tqdm
import argparse
import random
from scipy.signal import convolve2d
from scipy.signal import find_peaks
from sklearn.metrics import *
import matplotlib.pyplot as plt

from utils.tools import EarlyStopping, adjust_learning_rate, visual
from utils.metrics import metric
import os

from my_utils import *
%matplotlib Inline

In [14]:
SEQ_LEN = 128
MAST_RATIO = 0.2
BATCH_SIZE = 128 ## Here batch size is equivelant to stride

data_train = Qiang_Anomaly(setting='train', window_size=SEQ_LEN, mask_ratio=MAST_RATIO, test_mode=False, is_abs=False)
data_valid = Qiang_Anomaly(setting='valid', window_size=SEQ_LEN, mask_ratio=MAST_RATIO, test_mode=False, is_abs=False)
data_test = Qiang_Anomaly(setting='test', window_size=SEQ_LEN, mask_ratio=MAST_RATIO, test_mode=False, is_abs=False)

In [15]:
data_train = ConcatDataset([data_train, data_valid])
dataloader_train = DataLoader(data_train, batch_size=BATCH_SIZE, shuffle=False, num_workers=1, collate_fn=collate_fn)
dataloader_test = DataLoader(data_test, batch_size=BATCH_SIZE, shuffle=False, num_workers=1, collate_fn=collate_fn)

In [39]:
for i in range(len(data_train)):
    x, y, index_start, index_end, mask_size, meta = data_train.__getitem__(i)
    z = np.concatenate([x, y], axis=1) # 1250 x 128
    break

In [40]:
z.shape

(1250, 128)

In [34]:
x

array([[-0.24678914, -0.3776192 ,  0.9658221 , ...,  2.4302008 ,
        -0.46970293,  1.3147603 ],
       [ 0.3951734 ,  0.6369433 , -0.57845384, ...,  0.44198325,
        -0.48906913,  1.2388105 ],
       [ 0.4352574 ,  0.561785  ,  0.19059487, ...,  0.12780744,
        -0.9111417 , -0.7666372 ],
       ...,
       [ 2.976631  , -1.1930891 , -1.2396473 , ..., -1.0312737 ,
         1.0484477 , -0.11055335],
       [-1.8083684 , -1.0036275 , -0.02809047, ...,  0.09692855,
         2.2451024 , -1.191607  ],
       [-1.2881293 ,  0.48657486, -0.1956933 , ...,  0.18816543,
         0.042319  , -0.13617405]], dtype=float32)